In [1]:
using AMQPClient

## Connecting

In [2]:
AMQPClient.AMQP_DEFAULT_PORT

5672

In [3]:
AMQPClient.DEFAULT_AUTH_PARAMS

Dict{String,Any} with 3 entries:
  "MECHANISM" => "AMQPLAIN"
  "PASSWORD"  => "guest"
  "LOGIN"     => "guest"

In [4]:
connection(virtualhost="/",
        host="localhost", port=AMQPClient.AMQP_DEFAULT_PORT,
        auth_params=AMQPClient.DEFAULT_AUTH_PARAMS) do conn
    @info("connected!")
end

┌ Info: connected!
└ @ Main In[4]:4


## Opening a Channel

In [5]:
connection(virtualhost="/",
        host="localhost", port=AMQPClient.AMQP_DEFAULT_PORT,
        auth_params=AMQPClient.DEFAULT_AUTH_PARAMS) do conn
    channel(conn, AMQPClient.UNUSED_CHANNEL, true) do chan
        @info("channel opened")
    end
    @info("channel closed")
end

┌ Info: channel opened
└ @ Main In[5]:5
┌ Info: channel closed
└ @ Main In[5]:7


## Setting up Exchanges & Queues

In [6]:
function prepare_queue(chan)
    @info("declaring direct exchange named directexcg1")
    @assert exchange_declare(chan, "directexcg1", EXCHANGE_TYPE_DIRECT)

    @info("declaring queue named queue1")
    success, q_name, message_count, consumer_count = queue_declare(chan, "queue1")
    @assert success

    @info("binding queue to receive messages with routing key attribute route1")
    @assert queue_bind(chan, "queue1", "directexcg1", "route1")
end

function teardown_queue(chan)
    @info("unbinding queue from exchange")
    @assert queue_unbind(chan, "queue1", "directexcg1", "route1")

    @info("deleting queue")
    success, message_count = queue_delete(chan, "queue1")
    @assert success

    @info("deleting exchange")
    @assert exchange_delete(chan, "directexcg1")
end

connection(; virtualhost="/",
        host="localhost", port=AMQPClient.AMQP_DEFAULT_PORT,
        auth_params=AMQPClient.DEFAULT_AUTH_PARAMS) do conn
    channel(conn, AMQPClient.UNUSED_CHANNEL, true) do chan
        prepare_queue(chan)
        teardown_queue(chan)
    end
end

┌ Info: declaring direct exchange named directexcg1
└ @ Main In[6]:2
┌ Info: declaring queue named queue1
└ @ Main In[6]:5
┌ Info: binding queue to receive messages with routing key attribute route1
└ @ Main In[6]:9
┌ Info: unbinding queue from exchange
└ @ Main In[6]:14
┌ Info: deleting queue
└ @ Main In[6]:17
┌ Info: deleting exchange
└ @ Main In[6]:21


## Send and Receive Messages

In [7]:
function send_recv_message(chan)
    data = convert(Vector{UInt8}, codeunits("hello world"))
    msg = Message(data, content_type="text/plain", delivery_mode=PERSISTENT)

    @info("publishing a message", data=String(copy(msg.data)))
    basic_publish(chan, msg; exchange="directexcg1", routing_key="route1")
    msg = basic_get(chan, "queue1", false)
    if msg !== nothing
        @info("got a message", data=String(copy(msg.data)))
        basic_ack(chan, msg.delivery_tag)
    end
end

connection(; virtualhost="/",
        host="localhost", port=AMQPClient.AMQP_DEFAULT_PORT,
        auth_params=AMQPClient.DEFAULT_AUTH_PARAMS) do conn
    channel(conn, AMQPClient.UNUSED_CHANNEL, true) do chan
        prepare_queue(chan)
        send_recv_message(chan)
        teardown_queue(chan)
    end
end

┌ Info: declaring direct exchange named directexcg1
└ @ Main In[6]:2
┌ Info: declaring queue named queue1
└ @ Main In[6]:5
┌ Info: binding queue to receive messages with routing key attribute route1
└ @ Main In[6]:9
┌ Info: publishing a message
│   data = hello world
└ @ Main In[7]:5
┌ Info: got a message
│   data = hello world
└ @ Main In[7]:9
┌ Info: unbinding queue from exchange
└ @ Main In[6]:14
┌ Info: deleting queue
└ @ Main In[6]:17
┌ Info: deleting exchange
└ @ Main In[6]:21


## Asynchronous Message Consumer

In [8]:
function send_recv_messages2(chan)
    received = false
    success, consumer_tag = basic_consume(chan, "queue1", (msg)->begin
        @info("got a message", data=String(copy(msg.data)))
        basic_ack(chan, msg.delivery_tag)
        received = true
    end)
    @assert success

    data = convert(Vector{UInt8}, codeunits("hello world"))
    msg = Message(data, content_type="text/plain", delivery_mode=PERSISTENT)    
    @info("publishing a message", data=String(copy(msg.data)))
    basic_publish(chan, msg; exchange="directexcg1", routing_key="route1")

    while !received
        sleep(1)
    end
    @info("cancelling consumer")
    basic_cancel(chan, consumer_tag)
end

connection(; virtualhost="/",
        host="localhost", port=AMQPClient.AMQP_DEFAULT_PORT,
        auth_params=AMQPClient.DEFAULT_AUTH_PARAMS) do conn
    channel(conn, AMQPClient.UNUSED_CHANNEL, true) do chan
        prepare_queue(chan)
        send_recv_messages2(chan)
        teardown_queue(chan)
    end
end

┌ Info: declaring direct exchange named directexcg1
└ @ Main In[6]:2
┌ Info: declaring queue named queue1
└ @ Main In[6]:5
┌ Info: binding queue to receive messages with routing key attribute route1
└ @ Main In[6]:9
┌ Info: publishing a message
│   data = hello world
└ @ Main In[8]:12
┌ Info: got a message
│   data = hello world
└ @ Main In[8]:4
┌ Info: cancelling consumer
└ @ Main In[8]:18
┌ Info: unbinding queue from exchange
└ @ Main In[6]:14
┌ Info: deleting queue
└ @ Main In[6]:17
┌ Info: deleting exchange
└ @ Main In[6]:21
